# Random Forest

~a.k.a 묻지마모델~

Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks that operates by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees.

(출처: https://en.wikipedia.org/wiki/Random_forest)

To quote from The Elements of Statistical Learning, <b>"Trees have one aspect that prevents them from being the ideal tool for predictive learning, namely inaccuracy."</b>

$\rightarrow$ In other words, they work great with the data used to create them, but <b>they are not flexible when it comes to classifying new samples.</b>

<span style="color:blue">Step 1. Create a bootstrap dataset</span>

<b>Original Dataset</b>
<img src="img/RF_01_class.PNG">

Original dataset과 같은 크기의 Bootstrap dataset을 만들기 위해서 <b>임의로(Ramdomly) Original dataset에서 복원추출</b>을 할 것이다.

** Allow duplicate entries in the bootstrapped dataset.

<b>Bootstrapped Dataset</b>
<img src="img/RF_02_class.PNG">

<span style="color:blue">Step 2. Create a decision tree using the bootstrapped dataset, but only use a random subset of variables (or columns) at each step.</span>

Tree를 분할(split) 할 때 네 개의 독립변수(independent variable)를 모두 이용하지 않고 임의로 두 개만 선택을 한다.

"좋은 혈액순환 여부"와 "동맥경화 여부"를 선택되었고, "동맥경화 여부"가 분할에 가장 적합한 변수였다고 하면 해당 변수로 Tree를 분할 한다.

그러면 그 다음 또 다시 네 개의 독립변수 중 두 개를 다시 임의로 골라서 분할에 적합한 변수를 선택한다.

이 과정을 반복되면서 다양한 모양의 Tree를 키우게 된다.



최종적으로 Tree들을 만들게 되면 Classification을 할 데이터를 각 Tree에 넣어 결과를 받아본다. 가장 많은 Tree가 분류한 결과(Class)로 분류가 된다.

#### Out-Of-Bag(OOB) dataset
Bootstrap dataset에 들어가지 못한 데이터(entries)를 의미한다.

Tree를 구성하는데 쓰이지 않았기 때문에 OOB dataset으로 Random Forest의 성능을 평가할 수 있다.

OOB 데이터 중 잘못 분류된 비율을 <b>Out-Of-Bag Error</b>라고 한다.

다시 정리하자면,<br>
방금 전에는 한 step마다 독립변수 두 개를 임의로 추출하여 그 중 하나를 선택하는 것으로 결정하였지만 OOB Error를 구할 수 있게 됨으로써 세 개를 임의로 추출하는 방법과 비교하여 어느 것이 더 정확한 Random Forest를 만들 수 있는지 알 수 있게 되었다.

# Missing data and sample clustering

<b>Filled-in Missing Values</b>
<img src="img/RF_03_class.PNG">
물론 initial guess는 틀릴 수 있다.

이제 이 가정에 대한 평가를 진행할 것이다.

먼저 결측치가 보완된 dataset으로 Random Forest를 만든다.<br>
그리고 모든 데이터를 생성한 모든 Tree에 넣어 결과를 받아본다.

일단 첫 번째 Tree에 넣어본 결과....

<img src="img/RF_04_class.PNG">

데이터가 다음과 같이 분류되었다.<br>
특히 initial guess가 들어간 Entry 4는 Entry 3와 같은 Class로 분류되었다.

## 참고.

<b>B</b>ootstrapping the data plus using the <b>agg</b>regate to make a decision is called <b>"Bagging"</b>